In [ ]:
#Big update 10/2/2021
#rewrote trinkets into object/classes
    #trinkets no longer add their stats through the sim. rejoice!
    #added hex shrunken head
    #added naaru's shifting sliver.
#changed bloodlust to be multiplicative rather than additive
#added and fleshed out paddingtons mana modeling:
    #innervate
    #mana tide
    #shadow priest
    #spirit based regen
    #out of combat regen
    #runes
#fixed moonfire not properly being affected by moonfury and imp moonfire

import numpy
import pandas as pd
import math #Paddington - For sqrt in spirit calculation in future

class Trinket:
    def __init__(trinket, cd, ttype, shared, sharedcd,  duration, chance, effect, effect_amount):
        trinket.cd = cd
        trinket.ttype = ttype
        trinket.shared = shared
        trinket.sharedcd = sharedcd
        trinket.duration = duration
        trinket.chance = chance
        trinket.effect = effect
        trinket.effect_amount = effect_amount

Icon_of_the_Silver_Crescent = Trinket(120, "Active", True, 20, 20, 0, "SP", 155) #tested
Dark_Iron_Smoking_Pipe = Trinket(120, "Active",  True, 20, 20, 0, "SP", 155) #tested
Xiris_Gift = Trinket(90, "Active", True, 15, 15, 0, "SP", 150) #tested
Essence_of_Sapphiron = Trinket(120, "Active", True, 20, 20, 0, "SP", 130) #tested
Scryers_Bloodgem = Trinket(90, "Active", True, 15, 15, 0, "SP", 150) #tested
Vengeance_of_the_Illidari = Trinket(90, "Active", True, 15, 15, 0, "SP", 120) #tested
Hex_Shrunken_Head = Trinket(120, "Active", True, 20, 20, 0, "SP", 211)#tested
Shifting_Naaru_Sliver = Trinket(90, "Active", True, 15, 15, 0, "SP", 320)
Skull_of_Guldan = Trinket(120, "Active", True, 20, 20, 0, "Haste", 175) #tested
Sextant_of_Unstable_Currents = Trinket(45, "Proc", False, 0, 15, 0, "SP", 190) #tested
Eye_of_Magtheridon = Trinket(0, "Proc", False, 0, 10, 0, "SP", 170) #tested
Quagmirrans_Eye = Trinket(45, "Proc", False, 0, 6, 0, "Haste", 320) #tested
Ashtongue_Talisman_of_Equilibrium = Trinket(0, "Proc", False, 0, 8, 25, "SP", 150) #tested
Empty = Trinket(0, "Proc", False, 0, 0, 0, "SP", 0) #tested

# Define here your stats and environment
# Stats :
intel = 462
crit_score = 324
hit_score = 111 # 113
spellpower = 1245
haste = 145
spirit = 265
mp5 = 7
trinket_one = Skull_of_Guldan
trinket_two = Ashtongue_Talisman_of_Equilibrium

#Implemented Trinkets. You need to use the name EXACTLY as its written or it will throw an error.
#Empty - Use this for an Empty trinket slot.
#Icon_of_the_Silver_Crescent
#Hex_Shrunken_Head
#Dark_Iron_Smoking_Pipe
#Xiris_Gift
#Essence_of_Sapphiron
#Scryers_Bloodgem
#Vengance_of_The_Illidari
#Skull_of_Guldan
#Sextant_of_Unstable_Currents
#Eye_Of_Magtheridon
#Quagmirrans_Eye
#Ashtongue_Talisman_Of_Equilibrium
#Shifting_Naaru_Sliver

#Paddington - Added toggling of spells
casting_SF = True
casting_wrath = False
SF_rank = 8
is_MF = True
is_IS = False

is_csd = True # Chaotic Skyfire Diamond equiped
is_spellstrike = False
is_spellfire = False
is_bowis = True
is_motw = True
is_divine_spirit = True
is_spirit_scroll = False
is_arcane_brilliance = True #40 intellect +14 motw
is_totem_of_wrath = True
is_wrath_of_air = True
is_draenei_in_group = False
curse_of_elements = 1.13 #1.10
is_blessing_kings = True # 10% stats increased
is_crusader = True
is_sham_4_piece = False
is_flask = True
is_food = True
is_wizard_oil = True
#TODO Paddington - Add mana oil
is_mana_oil = False
is_twilight_owl = True
is_eye_of_night = True
is_T5 = False
is_T6_2 = True
is_T6_4 = True
is_drums = True
is_lust = True
lust_count = 1
lust_when = 100 # Percentage at which you want to receive the first bloodlust. 100 means the first lust goes off on engage. 
is_shadow_priest = False
shadow_priest_dps = 1500
is_mana_tide = False
mana_tide_timer = 120 #timer in seconds

#TODO Paddington - Add mana regeneration from cooldowns
is_manapot = True
is_destruction_potion = False #WARNING: the sim prioritizes using mana pots rather than destruction pots, so if you have both enabled it will use mana and not destruction. 
is_rune = True
is_innervate = False
is_mana_spring = True
is_spriest = False
is_judgment_of_wisdom = True
misery = 1.05

## Update stats
if is_motw:
    intel = intel + 18
    spirit = spirit + 18
if is_arcane_brilliance:
    intel = intel + 40 
if is_divine_spirit:
    spirit = spirit + 50
if is_totem_of_wrath:
    hit_score = hit_score + (3 * 12.6)
    crit_score = crit_score + (3 * 22.1) 
if is_draenei_in_group:
    hit_score = hit_score + 12.6 
if is_blessing_kings:
    intel = intel * 1.1
    spirit = spirit * 1.1
if is_crusader:
    crit_score = crit_score + (3 * 22.1) 
if is_wrath_of_air:
    spellpower = spellpower + 101
if is_sham_4_piece:
    spellpower = spellpower + 20
if is_flask:
    spellpower = spellpower + 80
if is_food:
    spellpower = spellpower + 23
    spirit = spirit + 20
if is_wizard_oil: 
    spellpower = spellpower + 36
    crit_score = crit_score + 14
if is_twilight_owl:
    crit_score = crit_score + (2 * 22.1)
if is_eye_of_night:
    spellpower = spellpower + 34    
if is_bowis:
    mp5 = mp5 + 49
if is_mana_spring:
    mp5 = mp5 + 50
if is_spirit_scroll and not is_divine_spirit:
    spirit = spirit + 30
if is_shadow_priest:
    mp5 = mp5 + (.05 * shadow_priest_dps)

# Functions
def logfunc(s):
    is_log_on = False
    if is_log_on:
        print(s)
        
def compute_dps(intel, crit_score, hit_score, spellpower, haste, is_csd,
                is_spellstrike, is_spellfire):
    print("Provided values : " + " int " + str(intel) + " crit " +  str(crit_score) + " hit " +  str(hit_score) + " sp " +  str(spellpower)+ " spirit " + str(spirit))
    
    fight_length = 300  # in seconds
    
    # Talents
    balance_of_power = 4 # +4% Hit
    focused_starlight = 4 # +4% crit for SF and Wrath
    moonkin_form = 5 # +5% Crit
    improved_mf = 10 # +10% Moonfire crit 
    imp_mf = 1.1 #10% increased moonfire damage
    starlight_wrath = True # reduce cast time by 0.5s
    vengeance = True # +100% Crit damange
    lunar_guidance = True # Spellpower bonus = 24% of total intel
    moonfury = 1.1 # +10% damage
    wrath_of_cenarius = 1.2 # +20% Spellpower for SF | +10% SpellPower for Wrath
    
    intensity = 3 #mana regen

    # Sets bonuses
    spellfire = is_spellfire # SP bonus = +7% of total intellect
    spellstrike = is_spellstrike # 5% chance to have +92sp for 10s - No ICD
    windhawk = False # 8MP/5 KEK

    # Meta GEM - Chaotic Skyfire Diamond
    csd_equiped = is_csd
    
    # Juff - making mana work
    start_mana = 2370 + intel*15
    mana = start_mana
    inn_amt = (((20 *.00932715221261 * math.sqrt(intel) * spirit)*(5-(intensity/10))))
    logfunc("inn_amt:" + str(inn_amt) + " start_mana: " + str(start_mana))
    
    #Paddington - Added mana regen
    #Juff - Fully Fleshed Out mana Regen
    #Mana Regen
    dreamstate = .1 * intel
    spirit_regen = 5 *.00932715221261 * math.sqrt(intel) * spirit * (.1 * intensity)
    fsr_spirit = 5 *.00932715221261 * math.sqrt(intel) * spirit
    casting_mp5 = mp5 + dreamstate + spirit_regen
    fsr_mp5 = mp5 + dreamstate + fsr_spirit
    logfunc("mp5:"  + " dreamstate : " + str(dreamstate) + " spirit_regen: " + str(spirit_regen) + " worn+buff mp/5: " + str(mp5) + " Five second rule mp5: " + str(fsr_mp5))
    mp5_tick= 5
    

    # Translating stats to %
    # At level 70, 22.1 Spell Critical Strike Rating increases your chance to land a Critical Strike with a Spell by 1%
    # At level 70, 12.6 Spell Hit Rating increases your chance to Hit with Spells by 1%. Hit cap is 202 FLAT (not including talents & buffs).
    # Druids receive 1% Spell Critical Strike chance for every 79.4 points of intellect.

    
    FF_mana_cost = 145
    
    #Paddington - changed spell damages to refelct in game damage.
    # Moonfire base damage : 305 to 357 Arcane damage and then an additional 600 Arcane damage over 12 sec.
    MF_coeff = 0.15
    MF_mana_cost = 450
    if is_T6_2:
        MF_coeff_dot = 0.65
    else:
        MF_coeff_dot = 0.52
    # Starfire base damage : Causes 550 to 647 Arcane damage -> 658 on average
    SF_coeff = 1

    #Paddington - Added spell downranking
    if SF_rank == 6:
        SF_mana_cost = 287
        SF_average_damage = 553.5
    elif SF_rank == 7:
        SF_mana_cost = 309
        SF_average_damage = 614.5
    else:
        SF_mana_cost = 337
        SF_average_damage = 658
        
    MF_average_damage = 397.5
    if is_T6_2:
        MF_average_dot_damage = 720
    else:
        MF_average_dot_damage = 600
    partial_coeff = 0.94 # For the moment, let's say that in average, partials get 6% damage reduction
    #sf_cast_time = 3
    #sf_cast_time_ng = 2.5
    #Paddington - Added Insect Swarm
    IS_coeff = 0.76
    IS_mana_cost = 175
    IS_average_dot_damage = 792
    
    wrath_coeff = 0.671
    wrath_mana_cost = 232
    wrath_average_damage = 448.5
    wrath_cast_time = 1.5
    wrath_cast_time_ng = 1

    #Paddington - Added mana potions
    if is_manapot:
        manapot_cd = 0
        manapot_up = True
    else:
        manapot_cd = 120
        manapot_up = False
    #Juff - Adding Runes and innervate
    if is_rune:
        rune_cd = 0
        rune_up = True
    else:
        rune_cd = 120
        rune_up = False
    if is_innervate:
        innervate_cd = 0
        innervate_up = True
    else:
        innervate_cd = 360
        innervate_up = False
    
    # Apply spell haste coefficients here
    # 15.77 Spell Haste Rating increases casting speed by 1%
    # % Spell Haste at level 70 = (Haste Rating / 15.77)
    # New Casting Time = Base Casting Time / (1 + (% Spell Haste / 100))
    spell_haste = haste / 15.77
    # sf_cast_time = 3 / (1 + (spell_haste/100))
    # sf_cast_time_ng = 2.5 / (1 + (spell_haste/100))
    # print("SF Cast time : " + str(sf_cast_time))
    # print("SF NG Cast time : " + str(sf_cast_time_ng))

    # Spell power calculation for fight SP + lunar guidance 
    if lunar_guidance:
        spellpower = spellpower + 0.25 * intel
    if spellfire:
        spellpower = spellpower + 0.07 * intel
    if is_divine_spirit:
        spellpower = spellpower + .1 * spirit
    
    # Hit chance
    # 12.6 Spell Hit Rating -> 1%
    hit_chance = min(99, 83 + (hit_score/12.6) + balance_of_power)
    hit_chance_percent_value = hit_chance / 100
    logfunc("Hit chance is : " + str(hit_chance))

    # Crit chance
    # At level 70, 22.1 Spell Critical Strike Rating -> 1%
    # Druids receive 1% Spell Critical Strike chance for every 79.4 points of intellect.
    MF_crit_percent = crit_score/22.1 + intel/79.4 + improved_mf + moonkin_form 
    MF_crit_percent_value = MF_crit_percent / 100
    logfunc("Moonfire crit chance is : " + str(MF_crit_percent))
    if is_T6_4 == True:
        SF_crit_percent =  crit_score/22.1 + intel/79.4 +  + moonkin_form + focused_starlight + 5
    else: 
        SF_crit_percent =  crit_score/22.1 + intel/79.4 +  + moonkin_form + focused_starlight
    SF_crit_percent_value = SF_crit_percent / 100
    logfunc("Starfire crit chance is : " + str(SF_crit_percent))
    
    wrath_crit_percent =  crit_score/22.1 + intel/79.4 +  + moonkin_form + focused_starlight 
    wrath_crit_percent_value = wrath_crit_percent / 100
    logfunc("Wrath crit chance is : " + str(wrath_crit_percent))
    
    logfunc("Spellpower is  : " + str(spellpower))
 
    # Crit coeff
    if csd_equiped:
        crit_coeff = 2.09
    else:
        crit_coeff = 2
    
    # Spellstrike bonus:
    if spellstrike:
        spellstrike_bonus = 92
    else:
        spellstrike_bonus = 0
    
    # Prepare and launch the simulations
    loop_size = 1# number of fights simulated, Paddington - Changed to 1 and use 2nd cell for number of fights
    average_dps = 0
    n = 0
    while n < loop_size:
        n = n + 1
        # Initialization
        loopcount = 0
        total_damage_done = 0
        damage = 0
        fight_time = 0
        spellstrike_uptime = 0
        trinket_one_duration = 0
        trinket_one_cd = 0
        trinket_shared_cd = 0
        trinket_two_duration = 0
        trinket_two_cd = 0
        trinket_one_active = False
        trinket_two_active = False
        Mag_Triggered = False
        ff_uptime = 0
        mf_uptime = 0
        is_uptime = 0
        trinket_uptime = 0
        trinket_cd_timer = 0
        is_trinket_active = False
        mana = start_mana
        manapot_cd = 0
        rune_cd = 0
        innervate_cd = 0 
        fight_spell_power = spellpower
        pre_loop_mana = start_mana
        post_loop_mana = start_mana
        tide_timer = mana_tide_timer
        tide_tick = 0 
        ticks_remaining = 4 
        five_second_rule = 0
        wrath_fight_value = wrath_crit_percent_value
        sf_fight_value = SF_crit_percent_value
        mf_fight_value = MF_crit_percent_value
        destro_duration = 0
        destro_active = False
        
        
        
        #Paddington - Added on-use trinket internal cooldown
        on_use_icd_timer = 0
        is_trinket_available = True
        is_ff_up = False
        is_mf_up = False
        #Paddington - Added Insect Swarm
        is_is_up = False
        is_ng = False
        spellstrike_proc = False
        ng_proc = False
        drum_time = 0
        drum_cd = 0
        lust_amount = lust_count
        lust_uptime = 0
        lust_start = 1 - (lust_when/100)
        is_lusted = False
        fight_haste = spell_haste
        Ashtongue_Triggered = False
        
        #Paddington - Added individual spell damage totals
        SF_damage = 0
        MF_damage = 0
        IS_damage = 0
        Wrath_damage  = 0
        #Treant_damage = 0
        SF_hit = False
        MF_hit = False
        IS_hit = False
        Wrath_hit = False


        # Time to kick ass and chew bubble gum
        while fight_time <= fight_length:
            pre_loop_mana = mana
            logfunc("pre loop mana" + str(pre_loop_mana))
            if is_lusted:
                loop_duration = max(1, (1.5 / (1 + (fight_haste / 100))/1.3)) #GCD - can't be less, it's the rule !
            else:
                loop_duration = max(1, (1.5 / (1 + (fight_haste / 100)))) #GCD - can't be less, it's the rule !
            damage = 0
            # adding a variable to keep the initial spellpower to revert to this value in case a trinket bonus
            # fades out before the end of the SF / Wrath cast (ie. no spellpower snapshot)
            
            if spellstrike_proc:
                fight_spell_power = fight_spell_power + spell_strike_bonus
           
            if is_drums and drum_cd <= 0:
                drum_time = 30
                drum_cd = 120
                drums_up = True
                fight_haste = fight_haste + (80/15.77)
                logfunc(" drums up !!! fight haste is " + str(fight_haste))
                
            if is_lust and lust_amount >= 1 and fight_time >= (lust_start * fight_length) and is_lusted == False:
                lust_amount = lust_amount - 1
                lust_uptime = 40
                is_lusted = True
                logfunc("Lusted! : " + str(is_lusted))
                
            if is_manapot and manapot_up and mana < (start_mana-3000):
                mana = mana + (1800 + 3000)/2
                manapot_cd = 120
                manapot_up = False
                logfunc("Mana Potion Used!")
                
            if is_rune and rune_up and mana < (start_mana - 1500) and not manapot_up:
                mana = mana + (900+1500)/2
                rune_cd = 120
                rune_up = False
                logfunc("Rune used!")
            
            # if FF not up, cast FF
            #Paddington - Added mana management
            if not is_ff_up and mana >= FF_mana_cost:
                logfunc("Casting Faerie Fire !")
                #Paddington - Added mana management
                mana = mana - FF_mana_cost
                logfunc("Current mana:" + str(mana))
                if is_lusted:
                    loop_duration = max(1, ((1.5 / (1 + (fight_haste/100)))/1.3)) #GCD
                else:
                    loop_duration = max(1, (1.5 / (1 + (fight_haste/100)))) #GCD
                    
                is_crit = False # can't crit on FF
                
                damage = 0 # and no damage applied
                
                if(numpy.random.random() <= hit_chance_percent_value):
                    is_hit = True
                    ff_uptime = 40
                    is_ff_up = True
                    if is_hit and is_judgment_of_wisdom and numpy.random.randint(1, high = 3, size = 1) == 1:      
                        mana = mana + 74
                        logfunc("jowisdom proc! mana " + str(mana))
                    # Test if spellstrike is proc -> Commented out because tested later on in the code
                    # spellstrike_proc = (numpy.random.randint(1, high = 101, size = 1) <= 10)

                else:
                    is_hit = False
                    logfunc("Faerie Fire -> Resist !")

            # if Moonfire not up, cast Moonfire
            else:
                 # Once FF is up trigger trinket if available.
                if trinket_one.ttype == "Active" and trinket_one_cd <= 0 and trinket_one.shared and trinket_shared_cd <= 0 and trinket_one.effect == "SP":
                    logfunc("fight spell power is " + str(fight_spell_power))
                    fight_spell_power = fight_spell_power + trinket_one.effect_amount
                    trinket_one_duration = trinket_one.duration
                    trinket_one_cd = trinket_one.cd
                    trinket_shared_cd = trinket_one.sharedcd
                    trinket_one_active = True
                    logfunc("Trinket one active! fight spell power is " + str(fight_spell_power) + " Trinket cd " + str(trinket_one.cd) + " shared cd " + str(trinket_one.sharedcd) )
                    
                if trinket_one.ttype == "Active" and trinket_one_cd <= 0 and trinket_one.shared and trinket_shared_cd <= 0 and trinket_one.effect == "Haste":
                    fight_haste = fight_haste + (trinket_one.effect_amount/15.77)
                    trinket_one_duration = trinket_one.duration
                    trinket_one_cd = trinket_one.cd
                    trinket_shared_cd = trinket_one.sharedcd
                    trinket_one_active = True
                    logfunc("trinket one active! haste " + str(fight_haste))
                
                if is_destruction_potion and not is_manapot and manapot_cd <= 0:
                    fight_spell_power = fight_spell_power + 120
                    wrath_fight_value = wrath_fight_value + .02
                    sf_fight_value = sf_fight_value + .02
                    mf_fight_value = mf_fight_value + .02
                    destro_duration = 15
                    manapot_cd = 120
                    destro_active = True
                    logfunc("destruction pot used! sp " + str(fight_spell_power) + " wrath crit " + str(wrath_fight_value) + " sf crit " + str(sf_fight_value) + " mf crit " + str(mf_fight_value))
                    

            #setup for the second trinket slot
                if trinket_two.ttype == "Active" and trinket_two_cd <= 0 and trinket_two.shared and trinket_shared_cd <= 0 and trinket_two.effect == "SP":
                    fight_spell_power = fight_spell_power + trinket_two.effect_amount
                    trinket_two_duration = trinket_two.duration
                    trinket_two_cd = trinket_two.cd
                    trinket_shared_cd = trinket_two.sharedcd
                    trinket_two_active = True
                    logfunc("trinket two active! spellpower " +  str(fight_spell_power))
    
                if trinket_two.ttype == "Active" and trinket_two_cd <= 0 and trinket_two.shared and trinket_shared_cd <= 0 and trinket_two.effect == "Haste":
                    fight_haste = fight_haste + (trinket_two.effect_amount/15.77)
                    trinket_two_duration = trinket_two.duration
                    trinket_two_cd = trinket_two.cd
                    trinket_shared_cd = trinket_two.sharedcd
                    trinket_two_active = True
                    logfunc("trinket two active! haste " + str(fight_haste))

                #Paddington - Added mana management
                if not is_mf_up and is_MF and mana >= MF_mana_cost:
                    logfunc("Casting Moonfire !")
                    #Paddington - Added mana management
                    mana = mana - MF_mana_cost
                    logfunc("Current mana: " + str(mana))
                    if is_lusted:
                        loop_duration = max(1, ((1.5 / (1 + (fight_haste/100)))/1.3)) #GCD
                    else:
                        loop_duration = max(1, (1.5 / (1 + (fight_haste/100)))) #GCD
                    # Is it a hit ?
                    if(numpy.random.random() <= hit_chance_percent_value):
                        is_hit = True
                        MF_hit = True
                        if is_hit and is_judgment_of_wisdom and numpy.random.randint(1, high = 3, size = 1) == 1:      
                            mana = mana + 74
                            logfunc("jowisdom proc! mana " + str(mana))                                      
                        # Is it a crit ?
                        is_crit = (numpy.random.random() <= mf_fight_value)
                        # Is it a partial ?
                        #if(numpy.random.randint(1, high = 101, size = 1) <= hit_chance):
                        #    damage = MF_average_damage + MF_coeff * fight_spell_power * partial_coeff
                        damage = MF_average_damage + MF_coeff * fight_spell_power * moonfury * imp_mf * curse_of_elements * misery
                        # Apply damage
                        if is_crit:
                            damage = damage * crit_coeff
            
                            # DoT :
                        if is_T6_2:
                            damage = damage + ((MF_average_dot_damage + (MF_coeff_dot * fight_spell_power) * min(15, (fight_length - fight_time - 1.5))/15) * moonfury * imp_mf * curse_of_elements * misery)
                            logfunc("Damage done : " + str(damage))
                        else:
                            damage = damage + ((MF_average_dot_damage + (MF_coeff_dot * fight_spell_power) * min(12, (fight_length - fight_time - 1.5))/12)* moonfury * imp_mf * curse_of_elements * misery)
                            logfunc("Damage done : " + str(damage))

                        # There is a Hit ! update model
                        if is_T6_2:
                            is_mf_up = True
                            mf_uptime = 15
                        else:
                            is_mf_up = True
                            mf_uptime = 12
                    else:
                        is_hit = False
                        logfunc("Moonfire -> Resist ! ")
                        
                #Paddington - Added insect swarm use
                elif not is_is_up and is_IS and mana>= IS_mana_cost:
                    logfunc("Casting Insect Swarm !")
                    mana = mana - IS_mana_cost
                    logfunc("Current mana:" + str(mana))
                    if is_lusted:
                        loop_duration = max(1, ((1.5 / (1 + (fight_haste/100)))/1.3)) #GCD
                    else:
                        loop_duration = max(1, (1.5 / (1 + (fight_haste/100)))) #GCD because we cast a spell
                    # Is it a hit ?
                    if(numpy.random.random() <= hit_chance_percent_value):
                        is_hit = True
                        #Paddington - Added hit check for individual spell dmg totals
                        IS_hit = True
                        if is_hit and is_judgment_of_wisdom and numpy.random.randint(1, high = 3, size = 1) == 1:      
                            mana = mana + 74
                            logfunc("jowisdom proc! mana " + str(mana))                                           
                        # DoT :
                        damage = damage + ((IS_average_dot_damage + (IS_coeff * fight_spell_power) * min(12, (fight_length - fight_time - 1.5))/12) * misery)
                        # There is a Hit ! update model
                        is_is_up = True
                        is_uptime = 12
                    else:
                        is_hit = False
                        logfunc("Insect Swarm -> Resist ! ")
                        
                #Juff - Added Innervate
                elif is_innervate and innervate_cd <= 0 and not manapot_up and not rune_up and mana < start_mana - inn_amt:
                    innervate_cd = 360
                    mana = mana + inn_amt
                    if mana > start_mana:
                        mana = start_mana
                        logfunc("Innervate Used ! Time: " +  str(fight_time))
                    if is_lusted:
                        loop_duration = max(1, ((1.5 / (1 + (fight_haste/100)))/1.3)) #GCD
                    else:
                        loop_duration = max(1, (1.5 / (1 + (fight_haste/100)))) #GCD because we cast a spell
                        
                #Paddington - Added wrath option here   
                #Paddington - Added mana management         
                elif casting_wrath and mana >= wrath_mana_cost:
                    # Cast Wrath
                    logfunc("Casting Wrath !")
                    mana = mana - wrath_mana_cost
                    # Is it a hit ?
                    if(numpy.random.randint(1, high = 101, size = 1) <= hit_chance):
                        is_hit = True
                        #Paddington - Added hit check for individual spell dmg totals
                        Wrath_hit = True
                        if is_judgment_of_wisdom and numpy.random.randint(1, high = 3, size = 1) == 1: 
                            mana = mana + 74
                            logfunc("jowisdom proc! mana " + str(mana))
                        # Is it a crit ?
                        is_crit = (numpy.random.random  <= wrath_fight_value)
                        # Is it a partial ?
                        if(numpy.random.randint(1, high = 101, size = 1) > hit_chance):
                            logfunc("Partial hit !")
                            damage = (wrath_average_damage + (wrath_coeff * fight_spell_power * wrath_of_cenarius * partial_coeff )) * moonfury * misery
                            # logfunc("Damage done : " + str(damage))
                        else:
                            damage = (wrath_average_damage + (wrath_coeff * fight_spell_power * wrath_of_cenarius )) * moonfury * misery
                            logfunc("Damage done : " + str(damage))
                        if is_crit:
                            damage = damage * crit_coeff

                    else:
                        is_hit = False
                        logfunc("Wrath -> Resist ! ")
                    if is_lusted:
                        wrath_cast_time = max(1, ((1.5 / (1 + (fight_haste/100)))/1.3))
                        loop_duration = max(1, wrath_cast_time)
                    elif is_ng and is_lusted:
                        wrath_cast_time_ng = max(1, ((1 / (1 + (fight_haste/100)))/1.3))
                        loop_duration = max(1, wrath_cast_time_ng)
                    elif is_ng:
                        wrath_cast_time_ng = max(1, (1 / (1 + (fight_haste/100))))
                        loop_duration = max(1, wrath_cast_time_ng)
                    else:
                        wrath_cast_time = max(1, (1.5 / (1 + (fight_haste/100))))
                        loop_duration = max(1, wrath_cast_time)
                    is_ng = False # Consume NG once wrath is cast

                #Paddington - Added mana management
                elif casting_SF and mana >= SF_mana_cost:
                    # Cast Starfire
                    logfunc("Casting Starfire !")
                    #Paddington - Added mana management
                    mana = mana - SF_mana_cost
                    logfunc("Current mana:" + str(mana))
                    # Computing loop duration
                    if is_lusted:
                        sf_cast_time = max(1, ((3 / (1 + (fight_haste/100)))/1.3))
                        loop_duration = max(1, sf_cast_time)
                    elif is_ng and is_lusted:
                        sf_cast_time_ng = max(1, ((2.5 / (1 + (fight_haste/100)))/1.3))
                        loop_duration = max(1, sf_cast_time_ng)
                    elif is_ng:
                        sf_cast_time_ng = max(1, (2.5 / (1 + (fight_haste/100))))
                        loop_duration = max(1, sf_cast_time_ng)
                    else:
                        sf_cast_time = max(1, (3 / (1 + (fight_haste/100))))
                        loop_duration = max(1, sf_cast_time)
                    is_ng = False # Consume NG once SF is cast  

                    # Is it a hit ?
                    # if(numpy.random.randint(1, high = 101, size = 1) <= hit_chance):
                    if(numpy.random.random() <= hit_chance_percent_value):
                        is_hit = True
                        #Paddington - Added hit check for individual spell dmg totals
                        SF_hit = True
                        if is_judgment_of_wisdom and numpy.random.randint(1, high = 3, size = 1) == 1: 
                            mana = mana + 74
                            logfunc("jowisdom proc! mana " + str(mana))
                            
                        if trinket_one == Ashtongue_Talisman_of_Equilibrium and numpy.random.randint(1, high = 101, size = 1) <= 25 and Ashtongue_Triggered == False:
                            trinket_one_active = True
                            fight_spell_power = fight_spell_power + trinket_one.effect_amount
                            trinket_one_duration = trinket_one.duration
                            logfunc("fresh ashtongue trigger. spellpower " + str(fight_spell_power) + " fight_time " + str(fight_time))
                            Ashtongue_Triggered = True
    
                        elif trinket_one == Ashtongue_Talisman_of_Equilibrium and numpy.random.randint(1, high = 101, size = 1) <= 25 and Ashtongue_Triggered == True:
                            trinket_one_duration = trinket_one.duration
                            logfunc("ashtongue refresh spellpower " + str(fight_spell_power) + " fight_time " + str(fight_time))

                        if trinket_two == Ashtongue_Talisman_of_Equilibrium and numpy.random.randint(1, high = 101, size = 1) <= 25 and Ashtongue_Triggered == False:
                            trinket_two_active = True
                            fight_spell_power = fight_spell_power + trinket_two.effect_amount
                            trinket_two_duration = trinket_two.duration
                            logfunc("fresh ashtongue trigger. spellpower " + str(fight_spell_power) + " fight_time " + str(fight_time))
                            Ashtongue_Triggered = True
    
                        elif trinket_two == Ashtongue_Talisman_of_Equilibrium and numpy.random.randint(1, high = 101, size = 1) <= 25 and Ashtongue_Triggered == True:
                            trinket_two_duration = trinket_two.duration
                            logfunc("ashtongue refresh spellpower " + str(fight_spell_power) + " fight_time " + str(fight_time))

                        # Is it a crit ?
                        is_crit = (numpy.random.random() <= sf_fight_value)
                        if is_crit:
                            logfunc("Starfire -> Crit ! ")
                        # Is it a partial ?
                        #if(numpy.random.randint(1, high = 101, size = 1) > hit_chance):
                        #    logfunc("Partial hit !")
                        #    damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius * partial_coeff )) * moonfury
                            # logfunc("Damage done : " + str(damage))

                        # Let's check if the cast is finished before trinket bonus, if not, we remove the relevant bonus :
                        
                        if spellstrike_proc and (spellstrike_uptime < loop_duration):
                            fight_spell_power = fight_spell_power - spellstrike_bonus 
                            logfunc("Spellstrike fades out to early ...")
                            
                        if trinket_one == Ashtongue_Talisman_of_Equilibrium and trinket_one_active and trinket_one_duration <= loop_duration: 
                            logfunc("fight spellpower + " + str(fight_spell_power))
                            fight_spell_power = fight_spell_power - trinket_one.effect_amount
                            trinket_one_active = False
                            logfunc("ashtongue fades early fight spellpower + " + str(fight_spell_power))
                            Ashtongue_Triggered = False
                        
                        elif trinket_one == Eye_of_Magtheridon and trinket_one_active and trinket_one_duration <= loop_duration: 
                            logfunc("fight spellpower + " + str(fight_spell_power))
                            fight_spell_power = fight_spell_power - trinket_one.effect_amount
                            trinket_one_active = False
                            logfunc("Eye of Mag fades early fight spellpower + " + str(fight_spell_power))
                            Mag_Triggered = False
            
                        elif trinket_one_active and trinket_one_duration <= loop_duration and  trinket_one.effect == "SP":
                            logfunc("fight spellpower + " + str(fight_spell_power))
                            fight_spell_power = fight_spell_power - trinket_one.effect_amount
                            trinket_one_active = False
                            logfunc("trinket fades early fight spellpower + " + str(fight_spell_power))
                
                        if trinket_two == Ashtongue_Talisman_of_Equilibrium and trinket_two_active and trinket_two_duration <= loop_duration:
                            logfunc("fight spellpower " + str(fight_spell_power))
                            fight_spell_power = fight_spell_power - trinket_two.effect_amount
                            trinket_two_active = False
                            logfunc("ashtongue fades early fight spellpower + " + str(fight_spell_power))
                            Ashtongue_Triggered = False
                            
                        elif trinket_two == Eye_of_Magtheridon and trinket_two_active and trinket_two_duration <= loop_duration:
                            logfunc("eye of mag fades early fight spellpower + " + str(fight_spell_power))
                            fight_spell_power = fight_spell_power - trinket_two.effect_amount
                            trinket_two_active = False
                            logfunc("trinket fades early fight spellpower + " + str(fight_spell_power))
                            Mag_Triggered = False
                            
                        elif trinket_two_active and trinket_two_duration <= loop_duration and  trinket_two.effect == "SP":
                            fight_spell_power = fight_spell_power - trinket_two.effect_amount
                            trinket_two_active = False
                            logfunc("trinket fades early fight spellpower + " + str(fight_spell_power))
                            
                        if is_destruction_potion and not is_manapot and destro_duration <= loop_duration and destro_active:
                            fight_spell_power = fight_spell_power - 120
                            wrath_fight_value = wrath_fight_value -.02
                            sf_fight_value = sf_fight_value - .02
                            mf_fight_value = mf_fight_value - .02
                            logfunc("destruction fades out early! sp " + str(fight_spell_power) + " wrath crit " + str(wrath_fight_value) + " sf crit " + str(sf_fight_value) + " mf crit " + str(mf_fight_value))
                            destro_active = False
                            
                        # end of trinket verification
                        #Paddington - Added check for Insect Swarm
                        if is_T5 and is_ng and mf_uptime >= sf_cast_time_ng and is_uptime >= sf_cast_time_ng:
                            damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius )) * moonfury * 1.1  * curse_of_elements * misery                          
                        elif is_T5 and mf_uptime >= sf_cast_time:
                            damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius )) * moonfury * 1.1  * curse_of_elements * misery                      
                        elif is_T5 and is_uptime >= sf_cast_time:
                            damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius )) * moonfury * 1.1 * curse_of_elements * misery
                        else:
                            damage = (SF_average_damage + (SF_coeff * fight_spell_power * wrath_of_cenarius )) * moonfury * curse_of_elements * misery                           
                        if is_crit:
                            damage = damage * crit_coeff
                            logfunc("Damage done : " + str(damage))
                    else:
                        is_hit = False
                        logfunc("Starfire -> Resist ! ")

                    
                    #Treant_damage = 0
                    #Individual spell damage totals
                    if SF_hit:
                        SF_damage = SF_damage + damage
                        SF_hit = False

                    if MF_hit:
                        MF_damage = MF_damage + damage
                        MF_hit = False

                    if IS_hit:
                        IS_damage = IS_damage + damage
                        IS_hit = False
  
                    if Wrath_hit:
                        Wrath_damage = Wrath_damage + damage
                        Wrath_hit = False
                    

            # Update time and model
            fight_time = fight_time + loop_duration
            ff_uptime = ff_uptime - loop_duration
            mf_uptime = mf_uptime - loop_duration
            is_uptime = is_uptime - loop_duration
            trinket_one_duration = trinket_one_duration - loop_duration
            trinket_one_cd = trinket_one_cd - loop_duration
            trinket_two_duration = trinket_two_duration - loop_duration
            trinket_two_cd = trinket_two_cd - loop_duration
            trinket_shared_cd = trinket_shared_cd - loop_duration
            destro_duration = destro_duration - loop_duration
            #Paddington - Added Internal CD for on-use trinkets
            spellstrike_uptime = spellstrike_uptime - loop_duration
            drum_time = drum_time - loop_duration
            drum_cd = drum_cd - loop_duration
            tide_tick = tide_tick - loop_duration
            #####################
            #Paddington - Added Mana management
            #Juffington - More mana management                                                                                                
            manapot_cd = manapot_cd - loop_duration
            rune_cd = rune_cd - loop_duration
            mp5_tick = mp5_tick - loop_duration
            innervate_cd = innervate_cd - loop_duration
            lust_uptime = lust_uptime - loop_duration
            #logfunc("innervate cd:" + str(innervate_cd))
            ######################

            # Check the timer on buffs / debuffs                
            if spellstrike_uptime <= 0:
                fight_spell_power = fight_spell_power - spellstrike_bonus
                
            if mf_uptime <= 0:
                is_mf_up = False
                logfunc("mf down!")
            
            if is_drums and drum_time <= 0 and drums_up:
                fight_haste = fight_haste - (80/15.77)
                drums_up = False
                #logfunc("drums down!!! fight haste is: " + str(fight_haste))
                
            if trinket_one == Ashtongue_Talisman_of_Equilibrium and trinket_one_duration <= 0 and trinket_one_active and trinket_one.effect == "SP":
                fight_spell_power = fight_spell_power - trinket_one.effect_amount
                trinket_one_active = False
                logfunc("ashtongue fades fight spellpower + " + str(fight_spell_power))
                Ashtongue_Triggered = False
                
            elif trinket_one == Eye_of_Magtheridon and trinket_one_duration <= 0 and trinket_one_active and trinket_one.effect == "SP":
                fight_spell_power = fight_spell_power - trinket_one.effect_amount
                trinket_one_active = False
                logfunc("Eye of Mag Fades fight spellpower + " + str(fight_spell_power))
                Mag_Triggered = False
                
            elif trinket_one_duration <= 0 and trinket_one_active and trinket_one.effect == "SP":
                fight_spell_power = fight_spell_power - trinket_one.effect_amount
                trinket_one_active = False
                logfunc("trinket one fades. fight spellpower + " + str(fight_spell_power))
                
            elif trinket_one_duration <= 0 and trinket_one_active and trinket_one.effect == "Haste":
                fight_haste = fight_haste - (trinket_one.effect_amount/15.77)
                trinket_one_active = False
                logfunc("trinket one fades. fight haste " + str(fight_haste))
            
            if trinket_two == Ashtongue_Talisman_of_Equilibrium and trinket_two_duration <= 0 and trinket_two_active and trinket_two.effect == "SP":
                fight_spell_power = fight_spell_power - trinket_two.effect_amount
                trinket_two_active = False
                logfunc("ashtongue two fight spellpower + " + str(fight_spell_power))
                Ashtongue_Triggered = False
                
            elif trinket_two == Eye_of_Magtheridon and trinket_two_duration <= 0 and trinket_two_active and trinket_two.effect == "SP":
                fight_spell_power = fight_spell_power - trinket_two.effect_amount
                trinket_one_active = False
                logfunc("eye of mag fades early. fight spellpower + " + str(fight_spell_power))
                Mag_Triggered = False                
            
            elif trinket_two_duration <= 0 and trinket_two_active and trinket_two.effect == "SP":
                fight_spell_power = fight_spell_power - trinket_two.effect_amount
                logfunc("trinket two fades. fight spellpower + " + str(fight_spell_power))
                trinket_two_active = False
                
            elif trinket_two_duration <= 0 and trinket_two_active and trinket_two.effect == "Haste":
                fight_haste = fight_haste - (trinket_two.effect_amount/15.77)
                trinket_two_active = False
                #logfunc("trinket two fades. fight haste" + str(fight_haste))
            
            if is_destruction_potion and not is_manapot and destro_duration <= 0 and destro_active:
                fight_spell_power = fight_spell_power - 120
                wrath_fight_value = wrath_fight_value -.02
                sf_fight_value = sf_fight_value - .02
                mf_fight_value = mf_fight_value - .02
                logfunc("destruction fades sp " + str(fight_spell_power) + " wrath crit " + str(wrath_fight_value) + " sf crit " + str(sf_fight_value) + " mf crit " + str(mf_fight_value))
                destro_active = False
                
            #Paddington - Added mana potion use
            if manapot_cd <= 0:
                manapot_up = True
                logfunc("Potion Available!")
            else:
                logfunc("Mana Potion Down!")
            #Juff Adding runes
            if rune_cd <= 0: 
                rune_up = True
                logfunc("Rune available")
            else:
                logfunc("Rune down!")
                
            #Juff Vate too. 
            if innervate_cd <= 0:
                innervate_up = True
                #logfunc("Vate up!")
            #else:
                #logfunc("VateDown!")
                                                                                                                                                                                                           
            #Paddington - Added mana management
            if mp5_tick <= 0 and five_second_rule >= 5:
                logfunc("premp5 mana " + str(mana))
                mana = mana + fsr_mp5
                logfunc("post 5sr mp5 " + str(mana))
                mp5_tick = 5 + mp5_tick #accounting for the fact that mp/5 ticks arent happening exactly every 5 seconds, each tick will adjust to happen as close to five seconds as possible. 
            elif mp5_tick <= 0 and five_second_rule <= 5:
                logfunc("premp5 mana " + str(mana))
                mana = mana + casting_mp5
                logfunc("post intensity mana" + str(mana) + " fight time " + str(fight_time))
                mp5_tick = 5 + mp5_tick
                #logfunc("mp5 tick: " + str(mp5_tick) + " mana " + str(mana) + " fight time " + str(fight_time))
                
            if is_lusted and lust_uptime <= 0:
                is_lusted = False
                #logfunc("lust down!")
    
            # Update nature's grace
            if is_crit:
                if trinket_one == Sextant_of_Unstable_Currents and trinket_one_cd <= 0 and numpy.random.randint(1, high = 6, size = 1) == 1:
                        fight_spell_power = fight_spell_power + trinket_one.effect_amount
                        trinket_one_duration = trinket_one.duration
                        trinket_one_cd = trinket_one.cd
                        trinket_one_active = True
                        #logfunc("sextant active! spellpower " + str(fight_spell_power))
        
                if trinket_two == Sextant_of_Unstable_Currents and trinket_two_cd <= 0 and numpy.random.randint(1, high = 6, size = 1) == 1:
                        fight_spell_power = fight_spell_power + trinket_two.effect_amount
                        trinket_two_duration = trinket_two.duration
                        trinket_two_cd = trinket_two.cd      
                        trinket_two_active = True
                        #logfunc("sextant active! spellpower " + str(fight_spell_power))
                        
                is_ng = True

            total_damage_done = total_damage_done + damage 

            # If there is a Hit, Check if spellstrike / Quag'eye is proc or refreshed :
            if is_hit: 
                if is_spellstrike and numpy.random.randint(1, high = 11, size = 1) == 10:
                    spellstrike_proc = True
                    spellstrike_uptime = 10
                    logfunc("Spellstrike proc !!!")
                    
                #Quagmirrans_Eye setup

                if trinket_one == Quagmirrans_Eye and trinket_one_cd <= 0 and numpy.random.randint(1, high = 11, size = 1) == 1:
                    fight_haste = fight_haste + (trinket_one.effect_amount/15.77)
                    logfunc("quags eye procced! " + str(fight_haste))
                    trinket_one_duration = trinket_one.duration
                    trinket_one_cd = trinket_one.cd
                    trinket_one_active = True
                    logfunc("trinket cd timer" + str(trinket_one_cd))
    
                if trinket_two == Quagmirrans_Eye and trinket_two_cd <= 0 and numpy.random.randint(1, high = 11, size = 1) == 1:
                    fight_haste = fight_haste + (trinket_two.effect_amount/15.77)
                    logfunc("quags eye procced! " + str(fight_haste))
                    trinket_two_duration = trinket_two.duration
                    trinket_two_cd = trinket_two.cd
                    trinket_two_active = True
                    
            # If there is a resist, check eye of mag proc :
                #Eye_Of_Magtheridon setup
            if not is_hit:
                if trinket_one == Eye_of_Magtheridon and Mag_Triggered == False:
                    fight_spell_power = fight_spell_power + trinket_one.effect_amount
                    trinket_one_active = True
                    trinket_one_duration = trinket_one.duration
                    Mag_Triggered = True
                   #logfunc("Eye of Mag Triggered! Time remaining " + str(trinket_one.duration) + " spell power "+ str(fight_spell_power))

                elif trinket_one == Eye_of_Magtheridon and Mag_Triggered == True:
                    trinket_one_duration = trinket_one.duration
                    #logfunc("Eye of Mag refreshed! Time remaining " + str(trinket_one.duration))
                    
                if trinket_two == Eye_of_Magtheridon and Mag_Triggered == False:
                    fight_spell_power = fight_spell_power + trinket_two.effect_amount
                    trinket_two_active = True
                    trinket_two_duration = trinket_two.duration
                    Mag_Triggered = True
                    #logfunc("Eye of Mag Triggered! Time remaining " + str(trinket_two.duration))

                elif trinket_two == Eye_of_Magtheridon and Mag_Triggered == False:
                    trinket_two_duration = trinket_two.duration
                    #logfunc("Eye of Mag refreshed! Time remaining " + str(trinket_two.duration))

            # Print output
            post_loop_mana = mana
            #logfunc("post loop mana" + str(post_loop_mana))
            if post_loop_mana >= pre_loop_mana:
                five_second_rule = five_second_rule + loop_duration
                #logfunc("fsr: " + str(five_second_rule))
            else:
                five_second_rule = 0
                #logfunc("fsr: " + str(five_second_rule))
                
            if is_mana_tide and fight_time >= tide_timer and tide_tick <= 0 and ticks_remaining >= 1:
                mana = mana + (start_mana * .06)
                tide_tick = 3 + tide_tick
                ticks_remaining = ticks_remaining - 1 
                logfunc("tide ticked! ticks remaining " + str(ticks_remaining) + " tide mana recovered " + str(.06 * start_mana))


                
            logfunc("Loop Duration : " + str(loop_duration))
            logfunc("Loop Damage : " + str(damage))
            if damage > 0:
                loopcount = loopcount + 1
            
        #print("damage loops: ", loopcount)  Used to track how many loops actually do damage in a fight i.e. before mana runs out.
        logfunc("Overall damage done : " + str(total_damage_done))
        #Paddington - Added individual spell damage totals
        #print("Starfire damage : ", str(SF_damage))
        #print("Starfall DPS: ", SF_damage/fight_time)
        #print("Percent Starfall damage: ", (SF_damage/total_damage_done)*100)
        #print("Wrath damage : ", str(Wrath_damage))
        #print("Wrath DPS: ", Wrath_damage/fight_time)
        #print("Percent Wrath damage: ", (Wrath_damage/total_damage_done)*100)
        #print("Moonfire damage : ", str(MF_damage))
        #print("Moonfire DPS: ", MF_damage/fight_time)
        #print("Percent Moonfire damage: ", (MF_damage/total_damage_done)*100)
        #print("Insect Swarm damage : ", str(IS_damage))
        #print("Insect Swarm DPS: ", IS_damage/fight_time)
        #print("Percent Insect Swarm damage: ", (IS_damage/total_damage_done)*100)
        logfunc("Overall DPS : "  + str(total_damage_done/fight_time)) # We use fight_time here in case SF lands after the fight_length mark
        average_dps = average_dps + (total_damage_done/fight_time)
    
    #Paddington - Changed to average dps. Seems more accurate.
    real_average_dps = average_dps / loop_size
    logfunc("Average DPS : " + str(real_average_dps))
    return(real_average_dps)
 
# Adapting the function to panda dataframes
v_compute_dps = numpy.vectorize(compute_dps)

In [ ]:
# Run this part to calculate the average dps value of your configuration
n = 0
all_dps = 0
loop_size = 1
damage = 0

while n < loop_size :
    n = n + 1
    # all_dps = all_dps + compute_dps(intel = 381, crit_score = 243, hit_score = 135, spellpower = 1162, haste = 0)
    all_dps = all_dps + compute_dps(intel, crit_score, hit_score, spellpower, haste,
                                    is_csd, is_spellstrike, is_spellfire)
average_dps = all_dps / loop_size
print("Average DPS with current configuration is : ", average_dps)


In [ ]:
# testing

intel = 468
crit_score = 397
hit_score = 195
spellpower = 964

n = 0
all_dps = 0
loop_size = 1
damage = 0

while n < loop_size :
    n = n + 1
    # all_dps = all_dps + compute_dps(intel = 381, crit_score = 243, hit_score = 135, spellpower = 1162, haste = 0)
    all_dps = all_dps + compute_dps(intel, crit_score, hit_score, spellpower, haste,
                                   is_csd, is_spellstrike, is_spellfire)
average_dps = all_dps / loop_size
print("Average DPS with current configuration is : ", average_dps)


In [ ]:
# This part is useful if you want to generate random values and find stats coefficients

#compute_dps(intel = 359, crit_score = 269, hit_score = 52, spellpower = 1102, haste = 0, 
#                                  is_csd = False, is_spellstrike = True, is_spellfire = True)
import timeit
print("Starting loop")
# Generate the dataset
dataset_size = 10000 # number of rows
start = timeit.default_timer()

# Adapting the function to panda dataframes
v_compute_dps = numpy.vectorize(compute_dps)

##
# Creating the randomized dataframe for Chaotic Skyfire Diamond
##
df_csd = pd.DataFrame(dict(
    intel = numpy.random.randint(300, high = 500, size = dataset_size),
    crit = numpy.random.randint(150, high = 450, size = dataset_size),
    hit = numpy.random.randint(110, high = 200, size = dataset_size),
    sp = numpy.random.randint(650, high = 1400, size = dataset_size),
    # haste = numpy.random.randint(1, high = 11, size = dataset_size),
    haste = 1,
    spellstrike = False,
    spellfire = True,
    CSD = True
))
print('Dataframe generated for CSD, adding dps for each row.')
df_csd['dps'] = v_compute_dps(df_csd.intel, df_csd.crit, df_csd.hit, df_csd.sp, df_csd.haste, 
                              df_csd.CSD, df_csd.spellstrike, df_csd.spellfire)
stop = timeit.default_timer()
print("End of loop. Duration: ", stop - start )  
df_csd.to_csv(path_or_buf='dps_generated-CSD_trinket-QuagNoMagth.csv')

In [ ]:
##
# Static values
##
start = timeit.default_timer()
df_spellstrike = pd.DataFrame(dict(
    intel = 450, 
    crit = numpy.random.randint(150, high = 320, size = dataset_size),
    hit = numpy.random.randint(110, high = 156, size = dataset_size),
    sp = numpy.random.randint(650, high = 1400, size = dataset_size),
    haste = numpy.random.randint(1, high = 11, size = dataset_size),
    spellstrike = True,
    spellfire = True,
    CSD = False
))
print('Dataframe generated for Spellstrike, adding dps for each row.')
df_spellstrike['dps'] = v_compute_dps(df_spellstrike.intel, df_spellstrike.crit, df_spellstrike.hit, df_spellstrike.sp, 
                                      df_spellstrike.haste, df_spellstrike.CSD, df_spellstrike.spellstrike, df_spellstrike.spellfire)
stop = timeit.default_timer()
print("End of loop. Duration: ", stop - start )  
df_spellstrike.to_csv(path_or_buf='./output/dps_generated-Spellstrike2.csv')

In [ ]:
df.to_csv(path_or_buf='./output/dps_generated.csv')

In [ ]:
# Comparison between CSD and Spellstrike
is_csd = True # Chaotic Skyfire Diamond equiped
is_spellfire = False

n = 0
all_dps = 0
loop_size = 100
while n < loop_size :
    n = n + 1
    # all_dps = all_dps + compute_dps(intel = 381, crit_score = 243, hit_score = 135, spellpower = 1162, haste = 0)
    all_dps = all_dps + compute_dps(intel = 379, crit_score = 324, hit_score = 111, spellpower = 1117, haste = 0,
                                   is_csd = True, is_spellstrike = True, is_spellfire = False)
average_dps = all_dps / loop_size
print("Average DPS wit CSD is : ", average_dps)

# Spellstrike
is_csd = False # Chaotic Skyfire Diamond equiped
is_spellfire = True
n = 0
all_dps = 0
loop_size = 100
while n < loop_size :
    n = n + 1
    all_dps = all_dps + compute_dps(intel = 364, crit_score = 229, hit_score = 152, spellpower = 1182, haste = 0,
                                   is_csd = False, is_spellstrike = True, is_spellfire = True)
average_dps = all_dps / loop_size
print("Average DPS wit Spellstrike is : ", average_dps)



In [ ]:

# Haste tests
# This part is useful if you want to generate random values and find stats coefficients


import timeit    
dataset_size = 101
start = timeit.default_timer()
df_haste = pd.DataFrame(dict(
    intel = 572,
    crit = 516,
    hit = 149,
    sp = 1516,
    haste = range(145,155),
    spirit = 366,
    spellstrike = False,
    spellfire = False,
    CSD = True


))

print('Dataframe generated for Haste testing, adding dps for each row.')
df_haste['dps'] = v_compute_dps(df_haste.intel, df_haste.crit, df_haste.hit, df_haste.sp, 
                                      df_haste.haste, df_haste.CSD, df_haste.spellstrike, df_haste.spellfire)
stop = timeit.default_timer()
print("End of loop. Duration: ", stop - start )  
df_haste.to_csv(path_or_buf='dps_generated-haste.csv')

In [ ]:
df_haste.to_csv(path_or_buf='dps_generated-haste.csv')